In [1]:
using Pkg
# Activate environment that has ProgressiveHedging installed
Pkg.activate("..")

  Activating project at `/lustre/eaglefs/projects/pvb/cju/ProgressiveHedging.jl`


In [2]:
# Pkg.instantiate()

In [3]:
using Distributed
const WORKERS = 1 # Change to > 1 to use parallel
if nworkers() < WORKERS
    diff = (nprocs() == nworkers() ? WORKERS : WORKERS - nworkers())
    println("Adding $diff worker processes.")
    Distributed.addprocs(diff)
    # Make sure these workers also have an environment with PH installed
    @everywhere using Pkg
    for w in workers()
        @spawnat(w, Pkg.activate(".."))
    end
end

@everywhere using ProgressiveHedging
@everywhere const PH = ProgressiveHedging
@everywhere using Ipopt
@everywhere using JuMP
@everywhere using Xpress

┌ Info: Xpress: Found license file /nopt/nrel/apps/xpressmp/8.13.0/bin/xpauth.xpr
└ @ Xpress /home/cju/.julia/packages/Xpress/xOQbX/src/license.jl:44
┌ Info: Xpress: Development license detected.
└ @ Xpress /home/cju/.julia/packages/Xpress/xOQbX/src/license.jl:89


In [4]:
using JuMP

In [5]:
@everywhere function create_model(
        scenario_id::PH.ScenarioID,
        key_word_arg::String="Default key word argument", kwargs...
        )
    #Define the economic dispatch (ED) model
    g_max = [1000, 1000];
    # Minimum power output of generators
    g_min = [0, 300];

    h_max = [100, 100];
    h_min = [0, 0];

    h_budget = 500 ;
    # Incremental cost of generators 
    c_g = [50, 100];
    # Fixed cost of generators
    c_g0 = [1000, 0]
    # Incremental cost of wind generators
    c_w = 50;
    # Total demand
    d = [1500, 1600];
    d1 = [2200, 2350];
    # Wind forecast
    w_f = [200.0, 150.0];

    scen1 = [1.0, 0.0]
    scen2 = [0.0, 1.0];
    
    ed = Model(Xpress.Optimizer)
    
    # Define decision variables    
    @variable(ed, 0 <= g[i = 1:2, t = 1:2] <= g_max[i]) # power output of generators
    @variable(ed, 0 <= h[i = 1:2, t = 1:2] <= h_max[i]) # power output of generators
    @variable(ed, 0 <= w[t = 1:2] <= w_f[t]) # wind power injection
    @variable(ed, 0 <= h_b[i = 1:2] <= h_budget) # power output of generators

    # Define the objective function
    @objective(ed, Min, sum(c_g[i]*(g[i, t] ) for i in 1:2, t in 1:2) + sum(c_w *w[t] for t in 1:2))

    # Define the constraint on the maximum and minimum power output of each generator
    @constraint(ed, [i = 1:2, t = 1:2], g[i, t] <= g_max[i]) #maximum
    @constraint(ed, [i = 1:2, t = 1:2], g[i, t] >= g_min[i]) #minimum

    @constraint(ed, [i = 1:2, t = 1:2], h[i, t] <= h_max[i]) #maximum
    @constraint(ed, [i = 1:2, t = 1:2], h[i, t] >= h_min[i]) #minimum
    
    # Define the constraint on the wind power injection
    if scenario_id == PH.scid(0)
        @constraint(ed, sum(h[i, t] for i in 1:2, t in 1:2) <= sum(h_b[i]*scen1[i] for i in 1:2))
    else
        @constraint(ed, sum(h[i, t] for i in 1:2, t in 1:2) <= sum(h_b[i]*scen2[i] for i in 1:2))
    end
    @constraint(ed, sum(h_b[i] for i in 1:2) == h_budget)
    
    # Define the constraint on the wind power injection
    @constraint(ed, [t = 1:2], w[t] <= w_f[t])

    # Define the power balance constraint
    if scenario_id == PH.scid(0)
        @constraint(ed, [t = 1:2], sum(g[:, t] + h[:, t]) + w[t] == d[t])
    else
        @constraint(ed, [t = 1:2], sum(g[:, t] + h[:, t]) + w[t] == d1[t])
    end
    
    vdict = Dict{PH.StageID, Vector{JuMP.VariableRef}}([PH.stid(1) => VariableRef[h_b...],
                                                        PH.stid(2) => vcat(VariableRef[g...], VariableRef[h...], VariableRef[w...]),
                                                        ])
    return JuMPSubproblem(ed, scenario_id, vdict)
end

# Solve the economic dispatch problem


In [6]:
function build_scen_tree()

    probs = [0.5, 0.5]
    
    tree = PH.ScenarioTree()
    
    for l in 1:2
        PH.add_leaf(tree, tree.root, probs[l])
    end
    return tree
end
;

In [7]:
ef_model = @time PH.solve_extensive(build_scen_tree(),
    create_model, 
    ()->Ipopt.Optimizer(),
    "Unused example string",
    opt_args=(print_level=0,)
)
println(ef_model)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

 14.835266 seconds (35.52 M allocations: 1.946 GiB, 7.67% gc time, 99.71% compilation time)
Min 25 g[1,1]_{0} + 25 g[1,2]_{0} + 50 g[2,1]_{0} + 50 g[2,2]_{0} + 25 w[1]_{0} + 25 w[2]_{0} + 25 g[1,1]_{1} + 25 g[1,2]_{1} + 50 g[2,1]_{1} + 50 g[2,2]_{1} + 25 w[1]_{1} + 25 w[2]_{1}
Subject to
 h_b[1]_{0,1} + h_b[2]_{0,1} = 500.0
 g[1,1]_{0} + g[2,1]_{0} + h[1,1]_{0} + h[2,1]_{0} + w[1]_{0} = 1500.0
 g[1,2]_{0} + g[2,2]_{0} + h[1,2]_{0} + h[2,2]_{0} + w[2]_{0} = 1600.0
 g[1,1]_{1} + g[2,1]_{1} + h[1,1]_{1} + h[2,1]_{1} + w[1]_{1} = 2200.0
 g[1,2]_{1} + g[2,2]_{1} + h[1,2]_{1} + h[2,2]_{1} + w[2]_{1} = 2350.0
 g[

In [8]:
println(JuMP.termination_status(ef_model))
println(JuMP.objective_value(ef_model))
for var in JuMP.all_variables(ef_model)
    println("$var = $(JuMP.value(var))")
end

LOCALLY_SOLVED
239999.999048545
g[1,1]_{0} = 1000.0000055098012
g[2,1]_{0} = 299.99999699999665
g[1,2]_{0} = 1000.0000099997995
g[2,2]_{0} = 315.93939272236827
h[1,1]_{0} = -9.79951877986144e-9
h[2,1]_{0} = -9.79951877986144e-9
h[1,2]_{0} = 67.03029763268624
h[2,2]_{0} = 67.03029814534618
w[1]_{0} = 199.99999750980083
w[2]_{0} = 150.00000149979954
h_b[1]_{0,1} = 134.0605957484835
h_b[2]_{0,1} = 365.9394042515165
g[1,1]_{1} = 1000.0000099997995
g[2,1]_{1} = 834.0605705125026
g[1,2]_{1} = 1000.0000099997995
g[2,2]_{1} = 1000.0000017268327
h[1,1]_{1} = 82.96970880678194
h[2,1]_{1} = 82.96970868111632
h[1,2]_{1} = 99.9999933867845
h[2,2]_{1} = 99.99999338678367
w[1]_{1} = 200.00000199979954
w[2]_{1} = 150.00000149979957


In [9]:
st = build_scen_tree()
(n, err, rerr, obj, soln, phd) = PH.solve(st,
                                          create_model,
                                          PH.ScalarPenaltyParameter(25.0), "Passed to constructor.",
                                          atol=1e-8, rtol=1e-12, max_iter=1000,
                                          report=10, # print residual info every 10 iterations
                                          )
println("Number of iterations: ", n)
println("L^2 error: ", err)
println(obj)

Initializing...
...launching workers...
...initializing subproblems...
Solving...
Iter:    0   AbsR: 2.915476e+02   RelR: 1.166190e+00   Xhat: 2.500000e+02   X: 1.500000e+02
Iter:   10   AbsR: 5.000580e-01   RelR: 1.647634e-03   Xhat: 5.000580e-01   X: 3.626890e-07
Iter:   20   AbsR: 5.000489e-01   RelR: 1.620898e-03   Xhat: 5.000489e-01   X: 1.206676e-06
Iter:   30   AbsR: 5.000303e-01   RelR: 1.594985e-03   Xhat: 5.000303e-01   X: 2.211612e-06
Iter:   40   AbsR: 5.000005e-01   RelR: 1.569852e-03   Xhat: 5.000005e-01   X: 3.341756e-06
Iter:   50   AbsR: 5.000177e-01   RelR: 1.545642e-03   Xhat: 5.000177e-01   X: 3.469677e-05
Iter:   60   AbsR: 5.000002e-01   RelR: 1.522063e-03   Xhat: 5.000002e-01   X: 1.850819e-06
Iter:   70   AbsR: 4.999809e-01   RelR: 1.499186e-03   Xhat: 4.999809e-01   X: 2.181757e-06
Iter:   80   AbsR: 4.999621e-01   RelR: 1.476987e-03   Xhat: 4.999621e-01   X: 2.816528e-06
Iter:   90   AbsR: 4.999368e-01   RelR: 1.455417e-03   Xhat: 4.999368e-01   X: 2.968058e-0

┌ Warning: Performed 1000 iterations without convergence. Consider increasing max_iter from 1000.
└ @ ProgressiveHedging /lustre/eaglefs/projects/pvb/cju/ProgressiveHedging.jl/src/algorithm.jl:540


In [10]:
aobj = PH.retrieve_aug_obj_value(phd)
println("Augmented Objective: ", aobj)
println("Difference: ", aobj - obj)

Augmented Objective: 240000.05325622193
Difference: 6.334311910904944e-5


In [11]:
soln

,variable,value,stage,scenarios
,String,Float64,Int64,String
1,h_b[1],147.839,1,"0,1"
2,h_b[2],352.161,1,"0,1"
3,"h[2,2]",100.0,2,0
4,"h[1,2]",47.8388,2,0
5,"g[2,1]",300.0,2,0
6,"h[2,1]",0.0,2,0
7,"h[1,1]",3.00801e-5,2,0
8,w[2],150.0,2,0
9,w[1],200.0,2,0
